In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [3]:
data = pd.read_csv('../../data/telecom_churn.csv')

In [4]:
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [5]:
data.drop(['State', 'Voice mail plan'], axis=1, inplace=True)

In [7]:
data['International plan'] = data['International plan'].map({'Yes': 1,'No': 0})

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
Account length            3333 non-null int64
Area code                 3333 non-null int64
International plan        3333 non-null int64
Number vmail messages     3333 non-null int64
Total day minutes         3333 non-null float64
Total day calls           3333 non-null int64
Total day charge          3333 non-null float64
Total eve minutes         3333 non-null float64
Total eve calls           3333 non-null int64
Total eve charge          3333 non-null float64
Total night minutes       3333 non-null float64
Total night calls         3333 non-null int64
Total night charge        3333 non-null float64
Total intl minutes        3333 non-null float64
Total intl calls          3333 non-null int64
Total intl charge         3333 non-null float64
Customer service calls    3333 non-null int64
Churn                     3333 non-null bool
dtypes: bool(1), float64(8), int64(9)
memory usage

In [9]:
y = data['Churn'].astype('int') #это целевой признак, привели его к типу int

In [10]:
x = data.drop('Churn', axis=1)

In [11]:
x.shape, y.shape

((3333, 17), (3333,))

In [20]:
from sklearn.model_selection import train_test_split, cross_val_score #требуется для разбиения 70/30
import numpy as np

In [13]:
x_train, x_valid, y_train, y_valid = train_test_split(x,y, test_size=0.3, random_state = 17) #test_size - доля тестовой выборки
#метод разобьет на 2 вектора и x и y, поэтому вначале 4 переменных 

In [16]:
x_train.shape, x_valid.shape

((2333, 17), (1000, 17))

In [18]:
first_tree = DecisionTreeClassifier(random_state=17)

In [19]:
cross_val_score(first_tree, x_train, y_train, cv=5)

array([ 0.9143469 ,  0.91220557,  0.92291221,  0.90772532,  0.91416309])

In [21]:
cross_val_score(first_tree, x_train, y_train, cv=5).mean()

0.91427061602227722

In [22]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
first_knn = KNeighborsClassifier()

In [26]:
cross_val_score(first_knn, x_train, y_train, cv=5).mean()

0.86712740439845226

## настраиваем макс глубину для дерева

In [29]:
from sklearn.model_selection import GridSearchCV
#поиск по решетке грид - решетка, сеарч - поиск, цс - кросс-валидация

In [30]:
tree_params = {'max_depth': np.arange(1,11), 'max_features':[.5, .7, 1.]}
#диапазн от 1 до 10, тут перечислить все параметры для перебора

In [31]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [33]:
%%time
tree_grid.fit(x_train, y_train)

Wall time: 12.3 s


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=17, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), 'max_features': [0.5, 0.7, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

теперь у нас есть обученный объект tree_grid

In [34]:
tree_grid.best_score_,  tree_grid.best_params_

(0.94170595799399914, {'max_depth': 6, 'max_features': 1.0})

tree = GridSearchCV
#param_grid - словарь что перебирать

In [39]:
knn_params = {'n_neighbors': range(5, 30, 5) }#+ list(range(50, 100, 10))}

In [40]:
knn_grid = GridSearchCV(first_knn, knn_params, cv=5)

In [41]:
%%time
knn_grid.fit(x_train, y_train);

Wall time: 1.9 s


GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': range(5, 30, 5)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [42]:
knn_grid.best_score_, knn_grid.best_params_

(0.8701243034719246, {'n_neighbors': 10})

In [43]:
tree_valid_pred = tree_grid.predict(x_valid)

In [44]:
tree_grid.score(x_valid, y_valid)

0.94599999999999995

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
accuracy_score(y_valid, tree_valid_pred)

0.94599999999999995

In [47]:
from sklearn.tree import export_graphviz

In [53]:
export_graphviz(tree_grid.best_estimator_, out_file='telecom_tree.dot', 
                feature_names=x.columns, filled=True)

In [54]:
!dir *.dot

 ’®¬ ў гбва®©бвўҐ C ­Ґ Ё¬ҐҐв ¬ҐвЄЁ.
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : A82F-A7B8

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Users\USR\mlcourse_open\jupyter_notebooks\topic03_decision_trees_knn

26.09.2017  21:34             8я272 telecom_tree.dot
               1 д ©«®ў          8я272 Ў ©в
               0 Ї Ї®Є  94я355я107я840 Ў ©в бў®Ў®¤­®


In [55]:
from skimage.io import imread

In [56]:
image = imread('telecom_tree.dot')

OSError: cannot identify image file <_io.BufferedReader name='telecom_tree.dot'>